In [1]:
import numpy as np 
import re
import keras
from keras import backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import pandas as pd

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv("../movieSentimentAnalysis/train.tsv", delimiter='\t')
test = pd.read_csv("../movieSentimentAnalysis/test.tsv", delimiter='\t')

In [3]:
print(train.columns.values)
print(train.head())

['PhraseId' 'SentenceId' 'Phrase' 'Sentiment']
   PhraseId  SentenceId                                             Phrase  \
0         1           1  A series of escapades demonstrating the adage ...   
1         2           1  A series of escapades demonstrating the adage ...   
2         3           1                                           A series   
3         4           1                                                  A   
4         5           1                                             series   

   Sentiment  
0          1  
1          2  
2          2  
3          2  
4          2  


In [4]:
train = train.drop(['PhraseId', 'SentenceId'], axis = 1)
test = test.drop(['PhraseId', 'SentenceId'], axis = 1)

In [5]:
print(train.head())

                                              Phrase  Sentiment
0  A series of escapades demonstrating the adage ...          1
1  A series of escapades demonstrating the adage ...          2
2                                           A series          2
3                                                  A          2
4                                             series          2


Naive method for Preprocessing input

In [ ]:
"""word_list = []
max_length_of_phrase = 0
for i in train['Phrase']:
    length = 0
    for j in i.strip().split():
        word_list.append(j)
        length+=1
    if length > max_length_of_phrase:
        max_length_of_phrase = length
print(max_length_of_phrase)
for i in test['Phrase']:
    length = 0
    for j in i.strip().split():
        word_list.append(j)
        length+=1
    if length > max_length_of_phrase:
        max_length_of_phrase = length
print(max_length_of_phrase)"""

In [ ]:
"""word_list = list(set(word_list))
print(len(word_list))"""

In [ ]:
"""word_dictionary = dict(zip(word_list, [i+1 for i in range(0, len(word_list))]))"""

In [ ]:
"""X_train = np.zeros((len(train), max_length_of_phrase))
print(max_length_of_phrase)
for i in range(0, len(train['Phrase'])):
    phrase = train['Phrase'][i]
    phrase_dem = []
    length = 0
    for word in phrase.strip().split():
        phrase_dem.append(dictionary[word])
        length+=1
    phrase_dem.extend([0 for j in range(0, max_length_of_phrase-length)])
    X_train[i] = phrase_dem
X_test = np.zeros((len(test), max_length_of_phrase))
for i in range(0, len(test['Phrase'])):
    phrase = test['Phrase'][i]
    phrase_dem = []
    length = 0
    for word in phrase.strip().split():
        phrase_dem.append(dictionary[word])
        length+=1
    phrase_dem.extend([0 for _ in range(0, max_length_of_phrase-length)])
    X_test[i] = phrase_dem"""

In [ ]:
"""y_train = np.zeros((len(train), 5))
for i in range(0, len(train)):
    y_i = np.zeros((5))
    y_i[train['Sentiment'][i]-1] = 1
    y_train[i] = y_i"""

Using tf-idf

In [6]:
max_words  = 1000
max_length = 150
X = train['Phrase']
y = train['Sentiment']
onehot_encoder = OneHotEncoder(sparse=False)
y = y.reshape(len(y), 1)
y = onehot_encoder.fit_transform(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [7]:
token   = Tokenizer(num_words = max_words)
token.fit_on_texts(X_train)
seq     = token.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(seq, maxlen = max_length)
seq     = token.texts_to_sequences(X_val)
X_val   = sequence.pad_sequences(seq, maxlen = max_length)

In [8]:
model = Sequential()
model.add(Dense(50, input_shape = (max_length, ), activation="relu"))
model.add(Dense(30, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(5, activation="softmax"))

In [9]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
model.fit(X_train, y_train, epochs = 10, batch_size=32)

Epoch 1/10
124848/124848 [==============================] - 27s 214us/step - loss: 1.3604 - acc: 0.5087
Epoch 2/10
124848/124848 [==============================] - 26s 205us/step - loss: 1.2588 - acc: 0.5122
Epoch 3/10
124848/124848 [==============================] - 26s 206us/step - loss: 1.2565 - acc: 0.5122
Epoch 4/10
124848/124848 [==============================] - 26s 209us/step - loss: 1.2492 - acc: 0.5122
Epoch 5/10
124848/124848 [==============================] - 26s 210us/step - loss: 1.2366 - acc: 0.5122
Epoch 6/10
124848/124848 [==============================] - 26s 209us/step - loss: 1.2328 - acc: 0.5122
Epoch 7/10
124848/124848 [==============================] - 27s 220us/step - loss: 1.2319 - acc: 0.5122
Epoch 8/10
124848/124848 [==============================] - 27s 215us/step - loss: 1.2306 - acc: 0.5122
Epoch 9/10
124848/124848 [==============================] - 26s 212us/step - loss: 1.2290 - acc: 0.5121
Epoch 10/10
124848/124848 [==============================] - 27s

In [11]:
model.evaluate(X_val, y_val)

31212/31212 [==============================] - 3s 81us/step


[1.2430429702597705, 0.5010572856593618]

In [12]:
def lstm():
    model = Sequential()
    model.add(Embedding(max_words, 16, input_length = max_length, name = 'embedding'))
    model.add(LSTM(64, return_sequences = True, name = 'lstm_1'))
    model.add(LSTM(32, return_sequences = False, name = 'lstm_2'))
    model.add(Dense(5, activation='sigmoid', name = 'dense_1'))
    return model

In [13]:
model = lstm()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 64)           20736     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 49,317
Trainable params: 49,317
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs = 5, batch_size = 128)

Epoch 1/5
124848/124848 [==============================] - 458s 4ms/step - loss: 1.1809 - acc: 0.5365
Epoch 2/5
124848/124848 [==============================] - 427s 3ms/step - loss: 1.0654 - acc: 0.5820
Epoch 3/5
124848/124848 [==============================] - 428s 3ms/step - loss: 1.0360 - acc: 0.5914
Epoch 4/5
124848/124848 [==============================] - 437s 4ms/step - loss: 1.0263 - acc: 0.5939
Epoch 5/5
124848/124848 [==============================] - 498s 4ms/step - loss: 1.0154 - acc: 0.5995


In [ ]:
model.evaluate(X_val, y_val)

14592/31212 [=============>................] - ETA: 49s